In [1]:
import pandas as pd
import numpy as np

from aif360.algorithms.preprocessing import DisparateImpactRemover,Reweighing,LFR
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric

from sklearn.ensemble import RandomForestClassifier
from aif360.datasets import CompasDataset, AdultDataset

from sklearn.metrics import f1_score

# TODO: change the import method
import sys
import os
repo_root = os.path.dirname(os.getcwd())
sys.path.insert(0, repo_root)
repair_folder = os.path.join(repo_root, "humancompatible", "repair")
sys.path.insert(0, repair_folder)
from humancompatible.repair.cost import *
from humancompatible.repair.coupling_utils import *
from humancompatible.repair.data_analysis import *
from humancompatible.repair.group_blind_repair import *
from humancompatible.repair.metrics import *




import os
path=os.path.dirname(os.getcwd())

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


# if you need "OptimPreproc"
from aif360.algorithms.preprocessing.optim_preproc import OptimPreproc
from aif360.algorithms.preprocessing.optim_preproc_helpers.distortion_functions\
            import get_distortion_adult
from aif360.algorithms.preprocessing.optim_preproc_helpers.opt_tools import OptTools


pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[Reductions]'
pip install 'aif360[Reductions]'
pip install 'aif360[inFairness]'
pip install 'aif360[Reductions]'


In [2]:
class Projpreprocess:
    
    def __init__(self,traindata,x_list,var_list,K,e):

        self.K=K
        self.e=e
        self.x_list=x_list
        self.var_list=var_list
      
        self.var_dim=len(var_list)
        self.arg_list=[elem for elem in var_list if elem not in x_list]
        self.train = traindata.copy()
        self.df = self.train.convert_to_dataframe()[0]
        self.pa = self.train.protected_attribute_names[0]
        self.pa_index = self.train.feature_names.index(pa)
        self.label_name = self.train.label_names[0]
        self.df=self.df.rename(columns={self.pa:'S',self.label_name:'Y'})

        self.df['W'] = self.train.instance_weights
        for col in self.var_list+['S','Y']:
            self.df[col]=self.df[col].astype('int64')
        self.df=self.df[var_list+['S','W','Y']]
        if len(x_list)>1:
            self.df['X'] = list(zip(*[self.df[c] for c in x_list]))
            self.x_range=sorted(set(self.df['X']))
            weight=list(1/(self.df[x_list].max()-self.df[x_list].min())) # because ranges of attributes differ
            self.C=c_generate_higher(self.x_range,weight)
        else:
            self.df['X']=self.df[x_list]
            self.x_range=sorted(set(self.df['X']))
            self.C=c_generate(self.x_range)
        self.df = self.df[self.arg_list+['X','S','Y','W']].groupby(by=self.arg_list+['X','S','Y'],as_index=False).sum()
        self.distribution_generator()
        
    def distribution_generator(self):
        bin=len(self.x_range)
        dist=rdata_analysis(self.df,self.x_range,'X')
        dist['v']=[(dist['x_0'][i]-dist['x_1'][i])/dist['x'][i] for i in range(bin)]
        dist['t_x']=dist['x'] # #dist['x'] #dist['x_0']*0.5+dist['x_1']*0.5 
        self.px=np.matrix(dist['x']).T
        self.ptx=np.matrix(dist['t_x']).T
        if np.any(dist['x_0']==0): 
            self.p0=np.matrix((dist['x_0']+1.0e-9)/sum(dist['x_0']+1.0e-9)).T
        else:
            self.p0=np.matrix(dist['x_0']).T 
        if np.any(dist['x_1']==0):
            self.p1=np.matrix((dist['x_1']+1.0e-9)/sum(dist['x_1']+1.0e-9)).T
        else:
            self.p1=np.matrix(dist['x_1']).T 
        self.V=np.matrix(dist['v']).T
        # self.tv_origin=sum(abs(dist['x_0']-dist['x_1']))/2
        # return px,ptx,V,p0,p1
    
    def coupling_generator(self,method,Theta=1e-2):
        print(Theta)
        if method == 'unconstrained':
            coupling=baseline(self.C,self.e,self.px,self.ptx,self.K)
        elif method == 'barycentre':
            coupling=baseline(self.C,self.e,self.p0,self.p1,self.K)
        elif method == 'partial':
            coupling=partial_repair(self.C,self.e,self.px,self.ptx,self.V,Theta,self.K)
        return coupling

    def preprocess(self,method,Theta=1e-2):
        coupling = self.coupling_generator(method,Theta)
        if len(self.x_list)>1:
            df_proj=projection_higher(self.df,coupling,self.x_range,self.x_list,self.var_list)
        else:
            df_proj=projection(self.df,coupling,self.x_range,self.x_list[0],self.var_list)
        df_proj = df_proj.groupby(by=self.arg_list+['X','S','Y'],as_index=False).sum()
        X=list(zip(*df_proj['X']))
        df_proj = df_proj.assign(**{self.x_list[i]:X[i] for i in range(len(self.x_list))})
        df_proj=df_proj.drop('X',axis=1)
        df_proj=df_proj.rename(columns={'S':self.pa,'Y':self.label_name})
        binaryLabelDataset = BinaryLabelDataset(
            favorable_label=0,
            unfavorable_label=1,
            df=df_proj.drop('W',axis=1), 
            label_names=self.train.label_names,
            protected_attribute_names=self.train.protected_attribute_names,
            privileged_protected_attributes=[np.array([1.0])],unprivileged_protected_attributes=[np.array([0.])])
        binaryLabelDataset.instance_weights = df_proj['W'].tolist()
        # return binaryLabelDataset.align_datasets(self.train)
        return self.train.align_datasets(binaryLabelDataset)

In [3]:
class Baselinepreprocess:

    def __init__(self,train,test):
        self.train = train
        self.test = test
        self.pa = train.protected_attribute_names[0]
        self.pa_index = train.feature_names.index(pa)
        self.prigroups = [{self.pa: 1}]
        self.unprigroups = [{self.pa: 0}]

    def preprocessing(self,method):
        test_tranf = self.test.copy()
        if method == 'RW':
            RW = Reweighing(privileged_groups = self.prigroups,unprivileged_groups = self.unprigroups) #DisparateImpactRemover(repair_level = 1)
            RW.fit(self.train)
            train_tranf = RW.transform(self.train)
        elif method == 'DIremover':
            di = DisparateImpactRemover(repair_level = 1,sensitive_attribute=pa)
            train_tranf = di.fit_transform(self.train)
            test_tranf = di.fit_transform(self.test)
        elif method == 'LFR':
            TR = LFR(privileged_groups = self.prigroups,unprivileged_groups = self.unprigroups,
                     Az = 1, Ax = 0.01, Ay = 1,verbose=0)
            TR = TR.fit(self.train)
            train_tranf = TR.transform(self.train)
            test_tranf = TR.transform(self.test)
        elif method == 'OP':
            optim_options = {
                "distortion_fun": get_distortion_adult,
                "epsilon": 0.05,
                "clist": [0.99, 1.99, 2.99],
                "dlist": [.1, 0.05, 0]
            }
            OP = OptimPreproc(OptTools, optim_options)
            OP = OP.fit(self.train)
            train_tranf = OP.transform(self.train, transform_Y=True)
        return train_tranf, test_tranf

    def prediction(self,method,para=None):
        test_tranf = self.test.copy()
        if method == 'origin':
            train_tranf = self.train
        elif method in ['RW','DIremover','LFR','OP']:
            train_tranf,test_tranf = self.preprocessing(method)
        else:
            K=200
            e=0.01
            var_list=self.train.feature_names.copy()
            var_list.remove(self.pa)
            projpre=Projpreprocess(self.train,para['x_list'],var_list,K,e)
            train_tranf=projpre.preprocess(method,para['Theta'])

        di=self.DisparateImpact(train_tranf)
        print('Disparate Impact of train',di)

        if method != 'LFR':
            X_train = np.delete(train_tranf.features, self.pa_index, axis=1)
            y_train = train_tranf.labels.ravel()
            weight_train = train_tranf.instance_weights
            model=RandomForestClassifier(max_depth=5).fit(X_train,y_train, sample_weight=weight_train)

            X_test = np.delete(test_tranf.features, self.pa_index, axis=1)
            y_pred = model.predict(X_test)
        else:
            y_pred = test_tranf.labels
        return y_pred,di
    
    def DisparateImpact(self,data):
        di = pd.DataFrame({'S':data.protected_attributes.ravel().tolist(),
            'Y':data.labels.ravel().tolist(),
            'W':list(data.instance_weights)},columns=['S','Y','W'])
        privileged = self.train.privileged_protected_attributes[0][0]
        unprivileged = self.train.unprivileged_protected_attributes[0][0]
        numerator=sum(di[(di['S']==unprivileged)&(di['Y']==data.favorable_label)]['W'])/sum(di[di['S']==unprivileged]['W'])
        denominator=sum(di[(di['S']==privileged)&(di['Y']==data.favorable_label)]['W'])/sum(di[di['S']==privileged]['W'])
        if numerator==denominator:
            return 1
        return numerator/denominator

    def assess(self,method,para=None):
        if para != None:
            y_pred,di_train = self.prediction(method,para)
        else:
            y_pred,di_train = self.prediction(method)
        y_test_pred = self.test.copy()
        y_test_pred.labels = y_pred

        di=self.DisparateImpact(y_test_pred)
        f1_macro = f1_score(self.test.labels, y_pred, average='macro',sample_weight=self.test.instance_weights)
        f1_micro = f1_score(self.test.labels, y_pred, average='micro',sample_weight=self.test.instance_weights)
        f1_weighted = f1_score(self.test.labels, y_pred, average='weighted',sample_weight=self.test.instance_weights)
        print('Disparate Impact of '+str(method),di)
        print('f1 macro of '+str(method),f1_macro)

        new_row=pd.Series({'DI of train':di_train,'DI':di,'f1 macro':f1_macro,'f1 micro':f1_micro,'f1 weighted':f1_weighted,'method':method})
        return new_row.to_frame().T

# Compas

In [4]:
pa = 'race'
label_map = {1.0: 'Did recid.', 0.0: 'No recid.'}
protected_attribute_maps = {1.0: 'Caucasian', 0.0: 'Not Caucasian'}
privileged_groups = [{pa: 1}]
unprivileged_groups = [{pa: 0}]
cd = CompasDataset(protected_attribute_names=[pa],privileged_classes=[['Caucasian'],[1]], 
                    metadata={'label_map': label_map,'protected_attribute_maps': protected_attribute_maps},
                    features_to_drop=['age', 'sex', 'c_charge_desc'])
train,test = cd.split([0.6], shuffle=True) #len(test.instance_names) = 2057
var_list = cd.feature_names.copy()
var_list.remove(pa)
var_dim=len(var_list)

# df_train = df.loc[train.instance_names,:].reset_index(drop=True)
# df_test = df.loc[test.instance_names,:].reset_index(drop=True)
df=cd.convert_to_dataframe()[0]
df=df.rename(columns={pa:'S',cd.label_names[0]:'Y'})
df['W'] = cd.instance_weights
for col in var_list+['S','Y']:
    df[col]=df[col].astype('int64')
df=df[var_list+['S','W','Y']]

tv_dist=dict()
for x_name in var_list:
    x_range_single=list(pd.pivot_table(df,index=x_name,values=['W'],observed=False)[('W')].index) 
    dist=rdata_analysis(df,x_range_single,x_name)
    tv_dist[x_name]=sum(abs(dist['x_0']-dist['x_1']))/2
    print(x_name, tv_dist[x_name])
x_list=[]
for key,val in tv_dist.items():
    if val>0.1:
        x_list+=[key]

juv_fel_count 0.03210337325453563
juv_misd_count 0.04323143324022939
juv_other_count 0.021763780679615215
priors_count 0.12622233191661625
age_cat=25 - 45 0.054431947619680315
age_cat=Greater than 45 0.13519019921101838
age_cat=Less than 25 0.08075825159133806
c_charge_degree=F 0.07840757396162046
c_charge_degree=M 0.07840757396162046


In [5]:
methods=['origin','RW','DIremover','LFR'] 
report=pd.DataFrame(columns=['DI of train','DI','f1 macro','f1 micro','f1 weighted','method'])
for ignore in range(10):
    # train val test 4:2:4
    train,test = cd.split([0.4], shuffle=True) 
    valid,test = test.split([0.3], shuffle=True)
    
    prepro = Baselinepreprocess(train,test)
    for method in methods:
        report = pd.concat([report,prepro.assess(method)], ignore_index=True)

report.to_csv(path+'/data/report_preprocess_compas_'+str(pa)+'.csv',index=None)

Disparate Impact of train 0.8508063904090156
Disparate Impact of origin 0.790151357857663
f1 macro of origin 0.6606396161144692
Disparate Impact of train 1
Disparate Impact of RW 0.7912403973955378
f1 macro of RW 0.6595996061700033
Disparate Impact of train 0.8508063904090156
Disparate Impact of DIremover 0.87822695035461
f1 macro of DIremover 0.6509420624812522
Disparate Impact of train 0.9184663961285563
Disparate Impact of LFR 0.9273086507129059
f1 macro of LFR 0.6512482451580937
Disparate Impact of train 0.8380490196078431
Disparate Impact of origin 0.7570159008877184
f1 macro of origin 0.6582753992914918
Disparate Impact of train 1
Disparate Impact of RW 0.7527235579207451
f1 macro of RW 0.6633554405301845
Disparate Impact of train 0.8380490196078431
Disparate Impact of DIremover 0.8918455074337427
f1 macro of DIremover 0.6609850417128611
Disparate Impact of train 0.9137245175848118
Disparate Impact of LFR 0.9644233899093234
f1 macro of LFR 0.6559355161020726
Disparate Impact of t

# Adult

In [6]:
def load_data(data_path,var_list,pa):
    column_names = ['age', 'workclass', 'fnlwgt', 'education',
                'education-num', 'marital-status', 'occupation', 'relationship',
                'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
                'native-country', 'Y']
    na_values=['?']
    pa_dict={'Male':1,'Female':0,'White':1,'Black':0}
    label_dict={'>50K.':1,'>50K':1,'<=50K.':0,'<=50K':0}
    train_path = os.path.join(data_path, 'adult.data')
    test_path = os.path.join(data_path, 'adult.test')
    train = pd.read_csv(train_path, header=None,names=column_names,
                    skipinitialspace=True, na_values=na_values)
    test = pd.read_csv(test_path, header=0,names=column_names,
                    skipinitialspace=True, na_values=na_values)
    messydata = pd.concat([test, train], ignore_index=True)[var_list+[pa,'Y']]
    messydata=messydata.rename(columns={pa:'S'})
    messydata['S']=messydata['S'].replace(pa_dict)
    messydata['Y']=messydata['Y'].replace(label_dict)
    messydata=messydata[(messydata['S']==0)|(messydata['S']==1)]
    for col in var_list+['S','Y']:
        messydata[col]=messydata[col].astype('int64')
    messydata['W']=1
    bins_capitalgain=[100,3500,7500,10000]
    bins_capitalloss=[100,1600,1900,2200]
    bins_age=[26,36,46,56]
    bins_hours=[21,36,46,61]

    messydata=categerise(messydata,'age',bins_age)
    messydata=categerise(messydata,'hours-per-week',bins_hours)
    messydata=categerise(messydata,'capital-gain',bins_capitalgain)
    messydata=categerise(messydata,'capital-loss',bins_capitalloss)
    
    return messydata

def categerise(df,col,bins):
    for i in range(len(bins)+1):
        if i == 0:
            df.loc[df[col] < bins[i], col] = i
        elif i == len(bins):
            df.loc[df[col] >= bins[i-1], col] = i
        else:
            df.loc[(df[col] >= bins[i-1])& (df[col] < bins[i]), col] = i        
    return df

def choose_x(var_list,messydata):
    tv_dist=dict()
    for x_name in var_list:
        x_range_single=list(pd.pivot_table(messydata,index=x_name,values=['W'])[('W')].index) 
        dist=rdata_analysis(messydata,x_range_single,x_name)
        tv_dist[x_name]=sum(abs(dist['x_0']-dist['x_1']))/2
    x_list=[]
    for key,val in tv_dist.items():
        if val>0.1:
            x_list+=[key]  
    return x_list,tv_dist

In [7]:
# TODO: change the obtaining method for the adult data
data_path='C://personal//work//repair//.venv//Lib//site-packages//aif360//data//raw//adult'

var_list=['hours-per-week','age','capital-gain','capital-loss','education-num'] #,'education-num'
pa='race'
favorable_label = 1
var_dim=len(var_list)

messydata = load_data(data_path,var_list,pa)
x_list,tv_dist = choose_x(var_list,messydata)
messydata=messydata.rename(columns={'S':pa})
cd=BinaryLabelDataset(
    favorable_label=1,
    unfavorable_label=0,
    df=messydata,label_names='Y',protected_attribute_names=[pa])
train,test = cd.split([0.4], shuffle=True) 
valid,test = test.split([0.3], shuffle=True)

In [14]:
para={'x_list':x_list,'Theta':1e-2}
methods=['origin','RW','DIremover','LFR'] 
report=pd.DataFrame(columns=['DI of train','DI','f1 macro','f1 micro','f1 weighted','method'])
for ignore in range(10):
    # train val test 4:2:4
    train,test = cd.split([0.4], shuffle=True) 
    valid,test = test.split([0.3], shuffle=True)
    
    prepro = Baselinepreprocess(train,test)
    for method in methods:
        report = pd.concat([report,prepro.assess(method)], ignore_index=True)

report.to_csv(path+'/data/report_preprocess_adult_'+str(pa)+'.csv',index=None)

Disparate Impact of train 0.47223910848631023
Disparate Impact of origin 0.43825807084041307
f1 macro of origin 0.694201107170793
Disparate Impact of train 1
Disparate Impact of RW 0.4341662543295398
f1 macro of RW 0.6863462888817584


KeyboardInterrupt: 

In [9]:
report

,DI of train,DI,f1 macro,f1 micro,f1 weighted,method
0,0.467542,0.489939,0.683935,0.819314,0.79216,origin
1,1,0.511456,0.690812,0.819007,0.794974,RW
2,0.467542,0.4965,0.688262,0.819622,0.794136,DIremover
3,0.790206,0.932976,0.685474,0.809626,0.788862,LFR
4,0.457948,0.441882,0.678863,0.815726,0.787174,origin
5,1,0.438133,0.676989,0.815213,0.786154,RW
6,0.457948,0.439819,0.687472,0.816495,0.791215,DIremover
7,0.757844,0.864616,0.700593,0.808704,0.793542,LFR
8,0.492511,0.465343,0.690802,0.822287,0.796149,origin
9,1.0,0.460183,0.69525,0.823159,0.798405,RW


In [11]:
report.to_csv(path+'/data/report_preprocess_compas_'+str(pa)+'_'+str(para['Theta'])+'.csv',index=None)

NameError: name 'para' is not defined

In [ ]:
Baselinepreprocess(train,test).assess('partial',para=para)

In [ ]:
Baselinepreprocess(train,test).assess('LFR')

In [ ]:
Baselinepreprocess(train,test).assess('RW')

In [ ]:
Baselinepreprocess(train,test).assess('DIremover')

In [ ]:
Baselinepreprocess(train,test).assess('origin')